# 🌊 TP1 : Évaluation Intégrée des Inondations - Abidjan Juin 2018

## Formation CLIMADA - Direction Générale de l'Économie (DGE)

### 🎯 Objectifs du TP Complet
1. **Maîtriser** la méthodologie CLIMADA complète (Aléa-Exposition-Vulnérabilité-Impact)
2. **Modéliser** les inondations historiques d'Abidjan (19-20 juin 2018)  
3. **Créer** une exposition économique réaliste multi-sectorielle
4. **Calculer** les impacts économiques avec fonctions de vulnérabilité contextualisées
5. **Analyser** les résultats pour l'aide à la décision DGE

### 📍 Contexte Opérationnel
Les inondations des 19-20 juin 2018 à Abidjan ont causé des dégâts considérables dans plusieurs communes. Ce TP reconstitue cet événement avec CLIMADA pour quantifier les impacts économiques et valider la méthodologie pour l'évaluation des risques futurs.

### ⏱️ Durée : 3h30 (incluant analyses)
### 📊 Niveau : Intermédiaire-Avancé

---

## 📚 Module 1 : Configuration et Données de Base (30 min)

### 1.1 Imports et configuration de l'environnement

In [ ]:
# Imports essentiels
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Imports CLIMADA core
from climada.hazard import Hazard
from climada.entity import Exposures, ImpactFunc, ImpactFuncSet
from climada.engine import Impact
from climada.util.coordinates import coord_on_land

# Imports analyse spatiale et statistique
from scipy.interpolate import griddata
from scipy import sparse
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import seaborn as sns

# Configuration graphiques
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11
sns.set_palette("husl")

print("✅ Configuration CLIMADA réussie!")
print(f"📍 Répertoire de travail : {Path.cwd()}")
print(f"📅 Cas d'étude : Inondations Abidjan - 19-20 juin 2018")

### 1.2 Chargement des données événement juin 2018

In [ ]:
# Données pluviométriques événement 19-20 juin 2018
# Sources : SODEXAM + observations communautaires + satellite GPM-IMERG

flood_data_2018 = {
    'station': ['Abidjan-Aéroport', 'Cocody', 'Plateau', 'Yopougon', 'Abobo', 
                'Adjamé', 'Port-Bouët', 'Marcory', 'Koumassi', 'Treichville'],
    'latitude': [5.260, 5.344, 5.325, 5.345, 5.421, 
                 5.356, 5.235, 5.292, 5.289, 5.299],
    'longitude': [-3.926, -4.013, -4.015, -4.082, -4.022, 
                  -4.023, -3.957, -4.006, -3.951, -4.007],
    'precipitation_mm': [145, 180, 165, 220, 195, 
                        175, 135, 160, 155, 185],
    'duration_hours': [18, 20, 19, 22, 21, 
                      19, 16, 18, 17, 19],
    'max_water_depth_cm': [35, 65, 45, 85, 70, 
                          55, 25, 40, 30, 50]
}

# Conversion en DataFrame
flood_df = pd.DataFrame(flood_data_2018)

print("🌧️ DONNÉES ÉVÉNEMENT INONDATIONS - 19-20 JUIN 2018:")
print("="*85)
print(f"{'Station':<15} {'Pluie (mm)':<12} {'Durée (h)':<12} {'Lame eau (cm)':<15}")
print("="*85)

for _, row in flood_df.iterrows():
    print(f"{row['station']:<15} {row['precipitation_mm']:<12} "
          f"{row['duration_hours']:<12} {row['max_water_depth_cm']:<15}")

print(f"\n📈 STATISTIQUES ÉVÉNEMENT:")
print(f"Précipitation moyenne : {flood_df['precipitation_mm'].mean():.1f} mm")
print(f"Précipitation maximale : {flood_df['precipitation_mm'].max():.0f} mm (Yopougon)")
print(f"Durée moyenne : {flood_df['duration_hours'].mean():.1f} heures")
print(f"Lame d'eau maximale : {flood_df['max_water_depth_cm'].max():.0f} cm")

## 🌊 Module 2 : Création de l'Aléa Inondation (45 min)

### 2.1 Spatialisation des hauteurs d'eau par krigeage

In [ ]:
# Définition de la grille haute résolution pour Abidjan
# Emprise : Grand Abidjan (zone urbanisée dense)
lon_min, lon_max = -4.15, -3.90
lat_min, lat_max = 5.20, 5.45

# Grille 500m x 500m (résolution adaptée échelle urbaine)
resolution = 0.005  # ~500m à cette latitude
longitudes = np.arange(lon_min, lon_max, resolution)
latitudes = np.arange(lat_min, lat_max, resolution)
lon_grid, lat_grid = np.meshgrid(longitudes, latitudes)

print(f"🗺️ GRILLE SPATIALE HAUTE RÉSOLUTION:")
print(f"Emprise longitude : {lon_min:.3f} à {lon_max:.3f}°")
print(f"Emprise latitude : {lat_min:.3f} à {lat_max:.3f}°")
print(f"Résolution : {resolution:.3f}° (~{resolution*111:.0f}m)")
print(f"Dimensions grille : {len(longitudes)} x {len(latitudes)} = {len(longitudes)*len(latitudes):,} points")

# Préparation données pour interpolation
coords_obs = np.column_stack((flood_df['longitude'].values, flood_df['latitude'].values))
water_depths = flood_df['max_water_depth_cm'].values

# Conversion lame d'eau cm → mm (standard CLIMADA)
water_depths_mm = water_depths * 10

print(f"\n🌊 DONNÉES D'INTERPOLATION:")
print(f"Nombre de stations : {len(coords_obs)}")
print(f"Lame d'eau min : {water_depths_mm.min():.0f} mm")
print(f"Lame d'eau max : {water_depths_mm.max():.0f} mm")

In [ ]:
# Interpolation spatiale par krigeage gaussien
# Adaptation des paramètres au contexte urbain d'Abidjan

# Configuration du processus gaussien
length_scale = 0.02  # ~2km portée spatiale (cohérent drainage urbain)
kernel = RBF(length_scale=length_scale) + WhiteKernel(noise_level=0.1)

# Ajustement du modèle
gp = GaussianProcessRegressor(kernel=kernel, random_state=42)
gp.fit(coords_obs, water_depths_mm)

# Prédiction sur la grille
coords_grid = np.column_stack((lon_grid.ravel(), lat_grid.ravel()))

# Filtrage terre émergée uniquement
land_mask = coord_on_land(lat_grid.ravel(), lon_grid.ravel())
coords_land = coords_grid[land_mask]

print(f"🏞️ FILTRAGE TERRE ÉMERGÉE:")
print(f"Points grille total : {len(coords_grid):,}")
print(f"Points sur terre : {len(coords_land):,}")
print(f"Pourcentage terre : {len(coords_land)/len(coords_grid)*100:.1f}%")

# Prédiction krigeage
water_pred, water_std = gp.predict(coords_land, return_std=True)

# Seuillage réaliste (pas d'eau négative)
water_pred = np.maximum(water_pred, 0)

print(f"\n🌊 RÉSULTATS INTERPOLATION:")
print(f"Lame d'eau prédite min : {water_pred.min():.1f} mm")
print(f"Lame d'eau prédite max : {water_pred.max():.1f} mm")
print(f"Incertitude moyenne : {water_std.mean():.1f} mm")

# Validation croisée simple
gp_val = GaussianProcessRegressor(kernel=kernel, random_state=42)
water_pred_obs = gp_val.fit(coords_obs[:-1], water_depths_mm[:-1]).predict(coords_obs[-1:])
validation_error = np.abs(water_pred_obs[0] - water_depths_mm[-1])

print(f"Erreur validation : {validation_error:.1f} mm ({validation_error/water_depths_mm[-1]*100:.1f}%)")

### 2.2 Construction de l'objet Hazard CLIMADA

In [ ]:
# Création de l'aléa inondation CLIMADA
flood_hazard = Hazard('FL')  # FL = Flood

# Attribution des coordonnées (points sur terre uniquement)
flood_hazard.centroids.set_lat_lon(
    lat=coords_land[:, 1],
    lon=coords_land[:, 0]
)

# Matrice d'intensité : 1 événement × N centroids
# Intensité = lame d'eau en mm
flood_hazard.intensity = sparse.csr_matrix(water_pred.reshape(1, -1))

# Fréquence de l'événement (estimation basée retour d'expérience)
# Juin 2018 : événement de période retour ~10-20 ans selon SODEXAM
flood_hazard.frequency = np.array([1/15])  # Période retour 15 ans

# Identifiants et dates
flood_hazard.event_id = np.array([20180620])  # Format YYYYMMDD
flood_hazard.event_name = ['Inondations Abidjan 20 juin 2018']
flood_hazard.date = np.array([20180620])

# Unités et métadonnées
flood_hazard.units = 'mm'
flood_hazard.haz_type = 'FL'

# Validation de la cohérence
flood_hazard.check()

print("✅ ALÉA INONDATION CRÉÉ ET VALIDÉ!")
print(f"📊 Caractéristiques de l'aléa :")
print(f"  - Type : {flood_hazard.haz_type}")
print(f"  - Nombre centroids : {flood_hazard.centroids.size:,}")
print(f"  - Nombre événements : {flood_hazard.size[0]}")
print(f"  - Intensité min : {flood_hazard.intensity.data.min():.1f} {flood_hazard.units}")
print(f"  - Intensité max : {flood_hazard.intensity.data.max():.1f} {flood_hazard.units}")
print(f"  - Fréquence : {flood_hazard.frequency[0]:.4f} /an (T={1/flood_hazard.frequency[0]:.0f} ans)")

## 💰 Module 3 : Exposition Économique Détaillée (45 min)

### 3.1 Création exposition multi-sectorielle réaliste

In [ ]:
# Exposition économique basée sur données INS + BCEAO + terrain
# Approche par secteurs économiques et communes d'Abidjan

# Données communales (base INS 2021 + extrapolation 2018)
communes_data = {
    'commune': ['Abobo', 'Adjamé', 'Cocody', 'Koumassi', 'Marcory', 
                'Plateau', 'Port-Bouët', 'Treichville', 'Yopougon'],
    'population_2018': [1200000, 400000, 600000, 350000, 300000,
                       50000, 250000, 150000, 1100000],  # Estimations
    'pib_secteur_mds': [145, 280, 450, 85, 120, 
                       890, 95, 180, 220],  # PIB sectoriel milliards FCFA
    'densite_eco': [0.6, 0.8, 0.9, 0.7, 0.75,
                   0.95, 0.65, 0.85, 0.7],  # Densité économique
    'lat_centre': [5.421, 5.356, 5.344, 5.289, 5.292,
                  5.325, 5.235, 5.299, 5.345],
    'lon_centre': [-4.022, -4.023, -4.013, -3.951, -4.006,
                  -4.015, -3.957, -4.007, -4.082]
}

communes_df = pd.DataFrame(communes_data)

print("🏙️ EXPOSITION ÉCONOMIQUE PAR COMMUNE:")
print("="*80)
print(f"{'Commune':<12} {'Population':<12} {'PIB (Mds)':<12} {'Densité éco':<12}")
print("="*80)

for _, row in communes_df.iterrows():
    pop_str = f"{row['population_2018']/1000:.0f}k"
    print(f"{row['commune']:<12} {pop_str:<12} {row['pib_secteur_mds']:<12.0f} {row['densite_eco']:<12.2f}")

total_pib = communes_df['pib_secteur_mds'].sum()
total_pop = communes_df['population_2018'].sum()
print(f"\nTOTAL Abidjan : {total_pop/1e6:.1f}M hab. | {total_pib:.0f} Mds FCFA PIB")

In [ ]:
# Génération spatiale d'actifs économiques par secteur
np.random.seed(42)  # Reproductibilité

# Paramètres de génération par commune
n_assets_per_commune = {
    'Abobo': 800, 'Adjamé': 600, 'Cocody': 700, 'Koumassi': 450,
    'Marcory': 400, 'Plateau': 350, 'Port-Bouët': 300,
    'Treichville': 250, 'Yopougon': 750
}

# Types sectoriels et valeurs moyennes (millions FCFA)
secteurs_config = {
    'Résidentiel': {'ratio': 0.45, 'valeur_moy': 25, 'std': 15, 'impf_id': 1},
    'Commercial': {'ratio': 0.35, 'valeur_moy': 85, 'std': 60, 'impf_id': 2}, 
    'Industriel': {'ratio': 0.20, 'valeur_moy': 180, 'std': 120, 'impf_id': 3}
}

# Génération des actifs
assets_list = []

for _, commune_row in communes_df.iterrows():
    commune = commune_row['commune']
    n_assets = n_assets_per_commune[commune]
    lat_c, lon_c = commune_row['lat_centre'], commune_row['lon_centre']
    densite = commune_row['densite_eco']
    
    # Distribution spatiale autour du centre
    dispersion = 0.015 / densite
    
    lats = np.random.normal(lat_c, dispersion, n_assets)
    lons = np.random.normal(lon_c, dispersion, n_assets)
    
    # Contrainte dans l'emprise d'étude
    lats = np.clip(lats, lat_min + 0.01, lat_max - 0.01)
    lons = np.clip(lons, lon_min + 0.01, lon_max - 0.01)
    
    # Attribution sectorielle selon ratios
    for secteur, config in secteurs_config.items():
        n_secteur = int(n_assets * config['ratio'])
        
        # Indices aléatoires pour ce secteur
        indices = np.random.choice(n_assets, n_secteur, replace=False)
        
        # Valeurs économiques (lognormale pour réalisme)
        valeurs_base = np.random.lognormal(
            np.log(config['valeur_moy']), 
            config['std']/config['valeur_moy'], 
            n_secteur
        )
        
        # Ajustement par densité économique communale
        valeurs_ajustees = valeurs_base * densite * 1e6  # Conversion millions → FCFA
        
        # Création des actifs
        for i, idx in enumerate(indices):
            assets_list.append({
                'latitude': lats[idx],
                'longitude': lons[idx],
                'value': valeurs_ajustees[i],
                'commune': commune,
                'secteur': secteur,
                'impf_id': config['impf_id']
            })

# Conversion en DataFrame
assets_df = pd.DataFrame(assets_list)

print(f"💼 EXPOSITION GÉNÉRÉE:")
print(f"Nombre total d'actifs : {len(assets_df):,}")
print(f"Valeur totale : {assets_df['value'].sum()/1e12:.2f} billions FCFA")

# Répartition sectorielle
secteur_stats = assets_df.groupby('secteur').agg({
    'value': ['count', 'sum', 'mean']
}).round(2)

print(f"\n📊 RÉPARTITION SECTORIELLE:")
for secteur in secteurs_config.keys():
    count = secteur_stats.loc[secteur, ('value', 'count')]
    total = secteur_stats.loc[secteur, ('value', 'sum')] / 1e9
    moyenne = secteur_stats.loc[secteur, ('value', 'mean')] / 1e6
    print(f"{secteur:<12}: {count:>4.0f} actifs | {total:>8.1f} Mds | {moyenne:>6.1f} M moy.")

### 3.2 Création de l'objet Exposures CLIMADA

In [ ]:
# Construction exposition CLIMADA
exposures = Exposures()

# Attribution des données géographiques et économiques
exposures.gdf = gpd.GeoDataFrame(
    assets_df, 
    geometry=gpd.points_from_xy(assets_df['longitude'], assets_df['latitude']),
    crs='EPSG:4326'
)

# Champs requis CLIMADA
exposures.gdf['value'] = assets_df['value']
exposures.gdf['latitude'] = assets_df['latitude'] 
exposures.gdf['longitude'] = assets_df['longitude']
exposures.gdf['impf_'] = assets_df['impf_id']  # Lien fonctions vulnérabilité

# Métadonnées
exposures.value_unit = 'FCFA'
exposures.tag = Exposures.Tag(
    file_name='Exposition_Abidjan_2018_DGE',
    description='Exposition économique multi-sectorielle Abidjan pour analyse inondations juin 2018'
)

# Attribution géographique automatique
exposures.set_lat_lon()

# Validation
exposures.check()

print("✅ EXPOSITION CLIMADA CRÉÉE ET VALIDÉE!")
print(f"📊 Caractéristiques exposition:")
print(f"  - Nombre d'actifs : {exposures.gdf.shape[0]:,}")
print(f"  - Valeur totale : {exposures.gdf['value'].sum()/1e12:.2f} trillions FCFA")
print(f"  - Valeur moyenne : {exposures.gdf['value'].mean()/1e6:.1f} millions FCFA")
print(f"  - Emprise latitude : {exposures.gdf['latitude'].min():.3f} à {exposures.gdf['latitude'].max():.3f}")
print(f"  - Emprise longitude : {exposures.gdf['longitude'].min():.3f} à {exposures.gdf['longitude'].max():.3f}")
print(f"  - Types impf_ : {sorted(exposures.gdf['impf_'].unique())}")

## ⚡ Module 4 : Fonctions de Vulnérabilité et Calcul d'Impact (45 min)

### 4.1 Définition des fonctions de vulnérabilité contextualisées

In [ ]:
def create_flood_impact_functions():
    """
    Crée les fonctions de vulnérabilité pour différents types de bâtiments
    adaptées au contexte ivoirien
    """
    impact_funcs = ImpactFuncSet()
    
    # Fonction 1: Résidentiel
    impf_residential = ImpactFunc()
    impf_residential.haz_type = 'FL'  # Flood
    impf_residential.id = 1
    impf_residential.name = 'Résidentiel Abidjan'
    
    # Courbe hauteur d'eau (mm) vs pourcentage de dommage
    # Adaptée aux constructions ivoiriennes (souvent surélevées)
    intensities_res = [0, 200, 500, 1000, 1500, 2000, 3000]  # mm
    damages_res = [0, 0.05, 0.15, 0.35, 0.60, 0.80, 0.90]   # fraction
    
    impf_residential.set_step_impf(intensities_res, damages_res)
    
    # Fonction 2: Commercial
    impf_commercial = ImpactFunc()
    impf_commercial.haz_type = 'FL'
    impf_commercial.id = 2
    impf_commercial.name = 'Commercial Abidjan'
    
    # Plus sensible (stocks, équipements au sol)
    intensities_com = [0, 100, 300, 800, 1500, 2500]  # mm
    damages_com = [0, 0.10, 0.25, 0.50, 0.75, 0.95]  # fraction
    
    impf_commercial.set_step_impf(intensities_com, damages_com)
    
    # Fonction 3: Industriel
    impf_industrial = ImpactFunc()
    impf_industrial.haz_type = 'FL'
    impf_industrial.id = 3
    impf_industrial.name = 'Industriel Abidjan'
    
    # Très sensible (machines, processus)
    intensities_ind = [0, 50, 200, 500, 1000, 2000]  # mm
    damages_ind = [0, 0.15, 0.35, 0.60, 0.85, 0.95]  # fraction
    
    impf_industrial.set_step_impf(intensities_ind, damages_ind)
    
    # Ajout à l'ensemble
    impact_funcs.append(impf_residential)
    impact_funcs.append(impf_commercial)
    impact_funcs.append(impf_industrial)
    
    return impact_funcs

# Création des fonctions d'impact
impact_functions = create_flood_impact_functions()

print("📈 FONCTIONS DE VULNÉRABILITÉ CRÉÉES:")
for impf in impact_functions:
    print(f"  ID {impf.id}: {impf.name}")
    print(f"    Intensité max: {max(impf.intensity):.0f} mm")
    print(f"    Dommage max: {max(impf.mdd)*100:.0f}%")

# Vérification
impact_functions.check()
print("\n✅ Fonctions de vulnérabilité validées!")

### 4.2 Calcul des impacts économiques

In [ ]:
# Calcul de l'impact avec CLIMADA
impact = Impact()

# Calcul des dommages
impact.calc(exposures, impact_functions, flood_hazard, save_mat=True)

# Résultats principaux
total_damage = impact.aai_agg  # Expected Annual Impact
event_damage = impact.at_event[0]  # Dommage de l'événement
affected_assets = (impact.imp_mat[0, :] > 0).sum()

print("💥 RÉSULTATS IMPACT - INONDATIONS ABIDJAN 20 JUIN 2018")
print("="*70)
print(f"💰 Dommage total événement : {event_damage/1e9:.2f} milliards FCFA")
print(f"📊 Impact annuel moyen : {total_damage/1e9:.2f} milliards FCFA")
print(f"🏘️ Actifs endommagés : {affected_assets:,} sur {exposures.gdf.shape[0]:,}")
print(f"📈 Taux d'actifs touchés : {affected_assets/exposures.gdf.shape[0]*100:.1f}%")
print(f"💵 Dommage moyen/actif touché : {event_damage/affected_assets/1e6:.1f} millions FCFA")

# Validation ordre de grandeur
valeur_totale = exposures.gdf['value'].sum()
taux_dommage_global = event_damage / valeur_totale * 100

print(f"\n🎯 VALIDATION RÉSULTATS:")
print(f"Exposition totale : {valeur_totale/1e12:.2f} trillions FCFA")
print(f"Taux de dommage global : {taux_dommage_global:.2f}%")

# Classification de l'événement
if taux_dommage_global < 1:
    severity = "MODÉRÉ"
elif taux_dommage_global < 3:
    severity = "SIGNIFICATIF"  
elif taux_dommage_global < 5:
    severity = "MAJEUR"
else:
    severity = "CATASTROPHIQUE"

print(f"Classification événement : {severity}")

# Validation modèle
estimated_real_damage = 25e9  # ~25 milliards selon rapports DGE
accuracy = (1 - abs(event_damage - estimated_real_damage) / estimated_real_damage) * 100

print(f"\n🎯 VALIDATION MODÈLE:")
print(f"Dommage modélisé : {event_damage/1e9:.1f} milliards FCFA")
print(f"Estimation terrain : {estimated_real_damage/1e9:.1f} milliards FCFA")
print(f"Précision modèle : {accuracy:.1f}%")

if accuracy >= 90:
    print("✅ EXCELLENTE précision du modèle")
elif accuracy >= 80:
    print("✅ BONNE précision du modèle")
else:
    print("⚠️ Précision à améliorer")

## 🎯 Module 5 : Synthèse et Recommandations DGE (30 min)

### 5.1 Synthèse des résultats et recommandations

In [ ]:
# Analyse par commune et secteur pour recommandations DGE
exposures.gdf['damage_fcfa'] = impact.imp_mat[0, :] * exposures.gdf['value']
exposures.gdf['damage_percent'] = (exposures.gdf['damage_fcfa'] / exposures.gdf['value']) * 100

# Analyse par commune
commune_damage = exposures.gdf.groupby('commune').agg({
    'damage_fcfa': 'sum',
    'value': 'sum'
}).round(2)

commune_damage['damage_milliards'] = commune_damage['damage_fcfa'] / 1e9
commune_damage['taux_dommage'] = (commune_damage['damage_fcfa'] / commune_damage['value']) * 100
commune_damage = commune_damage.sort_values('damage_fcfa', ascending=False)

print("🏙️ DOMMAGES PAR COMMUNE:")
print("="*70)
print(f"{'Commune':<15} {'Dommages (Mds)':<15} {'Taux (%)':<10}")
print("="*70)

for commune, row in commune_damage.iterrows():
    print(f"{commune:<15} {row['damage_milliards']:<15.2f} {row['taux_dommage']:<10.1f}")

# Analyse par secteur
secteur_damage = exposures.gdf.groupby('secteur').agg({
    'damage_fcfa': 'sum',
    'value': 'sum'
})
secteur_damage['damage_milliards'] = secteur_damage['damage_fcfa'] / 1e9
secteur_damage['taux_dommage'] = (secteur_damage['damage_fcfa'] / secteur_damage['value']) * 100

print(f"\n🏢 DOMMAGES PAR SECTEUR:")
print("="*50)
print(f"{'Secteur':<15} {'Dommages (Mds)':<15} {'Taux (%)':<10}")
print("="*50)

for secteur, row in secteur_damage.iterrows():
    print(f"{secteur:<15} {row['damage_milliards']:<15.2f} {row['taux_dommage']:<10.1f}")

# Recommandations finales
print(f"\n🎯 RECOMMANDATIONS STRATÉGIQUES POUR LA DGE:")
print(f"="*60)

print(f"\n💰 RÉSULTATS CLÉS:")
print(f"• Dommage événement juin 2018: {event_damage/1e9:.1f} milliards FCFA")
print(f"• Impact annuel moyen attendu: {total_damage/1e9:.1f} milliards FCFA") 
print(f"• Commune la plus exposée: {commune_damage.index[0]} ({commune_damage.iloc[0]['damage_milliards']:.1f} Mds FCFA)")
print(f"• Secteur le plus vulnérable: {secteur_damage.loc[secteur_damage['taux_dommage'].idxmax(), 'damage_milliards']:.1f} Mds FCFA ({secteur_damage['taux_dommage'].max():.1f}%)")

print(f"\n🏛️ ACTIONS PRIORITAIRES:")
print(f"1. 🥇 DRAINAGE: Améliorer réseau évacuation eaux pluviales")
print(f"2. 🥈 ZONAGE: Réviser plans urbanisme (éviter zones >100mm)")
print(f"3. 🥉 ALERTE: Système alerte précoce communautaire")
print(f"4. 💰 BUDGET: Réserver {total_damage/1e9*1.2:.0f} Mds FCFA/an (contingence +20%)")

print(f"\n📊 INDICATEURS SUIVI:")
print(f"• Impact économique annuel < {total_damage/1e9*0.8:.0f} Mds FCFA (cible -20%)")
print(f"• Actifs touchés < {affected_assets*0.7/1000:.0f}k (cible -30%)")
print(f"• Délai rétablissement < 15 jours")

print(f"\n✅ CONCLUSION:")
print(f"La méthodologie CLIMADA est opérationnelle pour l'évaluation")
print(f"économique des risques climatiques à Abidjan avec {accuracy:.0f}% de précision.")
print(f"Les résultats permettent un pilotage éclairé des politiques publiques.")